In [1]:
import redis
from imdb import IMDb

In [2]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)
pipe = r.pipeline()

In [3]:
from imdbpie import Imdb
imdb1 = Imdb()
imdb1 = Imdb(anonymize=True) # to proxy requests

# Creating an instance with caching enabled
# Note that the cached responses expire every 2 hours or so.
# The API response itself dictates the expiry time)
imdb1 = Imdb(cache=True)

In [4]:
top250 = imdb1.top_250()
dict_top250 = {}
for i in range(len(top250)):
    dict_top250[((top250[i]['tconst'].encode('utf-8'))[2:len(top250[i]['tconst'].encode('utf-8'))])] = (top250[i]['title'].encode('utf-8'))

In [5]:
# for movId, title in dict_top250.iteritems():
#     imdb2 = IMDb()
#     my = imdb2.get_movie(movId)
#     for castMember in my['cast'][0:10]:
#         full_person = imdb2.get_person(castMember.getID(), info=["filmography"])
#         pipe.execute()
#         try:
#             r[castMember['name'].encode('utf-8')]
#         except KeyError: 
#             try:
#                 pipe.set(castMember.getID(), (castMember['name'].encode('utf-8'), full_person["actress"]))
#             except:
#                 pipe.set(castMember.getID(), (castMember['name'].encode('utf-8'), full_person["actor"]))

cast_dict = {}

for movId, title in dict_top250.iteritems():
    cast_list = []
#     print movId
    imdb2 = IMDb()
    my = imdb2.get_movie(movId)
#     pipe.execute()
    for castMember in my['cast'][0:10]:
#         cast_dict[castMember['name'].encode('utf-8')] = castMember.getID();
            cast_list.append(castMember.getID());
#     print cast_list
#     pipe.set(movId, cast_list);        
    cast_dict[movId] = cast_list
print cast_dict

{'0061512': ['0000056', '0001421', '0134201', '0031398', '0238105', '0001510', '0886888', '0416378', '0940971', '0039435'], '0482571': ['0413168', '0000288', '0000323', '0005305', '0356017', '0424060', '2195817', '0000309', '0785227', '0204425'], '0018455': ['0639563', '0310980', '0515272', '0743017', '0531759', '0802684', '0936079', '0396768', '0092665', '0201576'], '0114709': ['0000158', '0000741', '0725543', '0001815', '0001728', '0001652', '0001633', '0606658', '0902184', '0582418'], '0083987': ['0001426', '0000298', '0002081', '0000024', '0002145', '0590055', '0000640', '0153182', '0297538', '0356847'], '2084970': ['1212722', '0461136', '0328828', '1239499', '1395602', '0063914', '0001097', '0835016', '3485153', '0329259'], '0071562': ['0000199', '0000380', '0000473', '0000134', '0001030', '0001735', '0833448', '0311155', '0819525', '0109175'], '0045152': ['0000037', '0640307', '0001666', '0353405', '0593612', '0001998', '0288830', '0001549', '0011625', '0017021'], '1049413': ['00

In [6]:
def test(movies):
    """input: list of 250 movies
        output: hashtable"""
    act_assoc_map = {}
    for movie in movies:
        l_act = movies.get(movie)
        for act in l_act:
            for act_id in range(len(l_act)):
                if act not in act_assoc_map.keys():
                    if l_act[act_id] != act:
                        act_assoc_map[act] = [l_act[act_id]]
                else:
                    if l_act[act_id] != act:
                        act_assoc_map[act].append(l_act[act_id])
    for actor in act_assoc_map.keys():
        act_assoc_map[actor] = list(set(act_assoc_map[actor]))
    return act_assoc_map

hashtable = test(cast_dict)
#print hashtable
for key, value in hashtable.iteritems():
    print str(value)
    pipe.execute();
    for i in value:
        pipe.lpush(key, i)

['0416694', '0285495', '0717372', '0000093', '0311563', '0005458', '0001125', '0001199', '0005068']
['0281313', '0402032', '0000450', '0577513', '1814260', '0001057', '0315417', '0051509', '2512626']
['0088127', '0000959', '0202002', '1617514', '0206257', '0161980', '0004695', '0001125', '0209962']
['0336475', '0706368', '0675490', '0831889', '0000348', '0000493', '0000054', '0002285', '0790064']
['0001695', '0219666', '0060904', '0000752', '0807580', '0000059', '0178259', '0178066', '0115558']
['0002034', '0335788', '0001017', '0001673', '0001012', '0842137', '0002011', '0275835', '0000842', '0925713', '0550855', '0912001', '0834754', '0001430', '0938231', '0003225', '0001181', '0801193', '0083081', '0832375', '0000314', '0275213', '0703600', '0943978', '0100758', '0863193', '0702798']
['0335788', '0000992', '0000075', '0002121', '0000638', '0087404', '0000017', '0001703', '0355095', '0001050', '0533247', '0604656', '0909930', '0481618', '0001961', '0000044', '0459252', '0001847']
['0

In [12]:
def bfs(graph, start, end):
    # maintain a queue of paths
    queue = []
    # push the first path into the queue
    queue.append([start])
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        # get the last node from the path
        node = path[-1]
        # path found
        if node == end:
            return path
        # enumerate all adjacent nodes, construct a new path and push it into the queue
        for adjacent in graph.get(node, []):
            new_path = list(path)
            new_path.append(adjacent)
            queue.append(new_path)

hashlist = bfs(hashtable, '0000138', '0000307')

In [13]:
for i in hashlist: #hashlist is the bfs output
    print imdb2.get_person(i)

Leonardo DiCaprio
Christoph Waltz
Brad Pitt
Helena Bonham Carter


In [8]:
len(hashtable.keys())

2083

In [9]:
for key, value in hashtable.iteritems():
    print len(value)

9
9
9
9
9
27
18
9
18
28
18
9
9
9
9
17
9
9
9
9
9
9
9
9
18
9
9
9
4
9
9
9
9
9
9
9
9
9
9
9
9
9
9
17
9
9
9
9
9
18
9
9
9
9
9
9
9
18
9
9
17
27
18
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
17
18
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
21
9
9
9
18
18
9
9
9
9
9
9
9
9
9
9
9
9
9
9
11
9
9
9
9
9
9
9
17
9
9
9
9
9
9
16
18
9
17
50
27
9
9
48
9
9
9
9
32
18
9
9
9
27
9
9
9
34
9
9
9
9
9
9
17
9
27
9
9
27
18
9
9
9
9
18
9
9
9
17
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
16
9
9
9
9
9
9
9
16
9
9
22
9
9
9
18
18
9
9
9
9
9
9
9
9
9
9
9
9
9
17
16
18
9
9
9
9
9
9
18
9
9
9
9
9
9
9
9
18
9
9
18
9
20
9
9
9
9
9
18
9
9
9
9
9
18
9
9
9
9
17
9
9
9
9
9
9
18
9
9
9
9
17
18
9
9
9
9
25
9
9
9
9
9
9
9
9
9
16
9
9
9
9
23
9
9
9
18
9
9
26
9
17
9
9
9
17
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
18
9
9
18
9
9
9
9
9
9
13
9
9
31
9
9
9
33
9
9
9
9
9
9
9
9
9
9
9
9
9
9
18
9
9
9
9
9
17
9
9
18
9
9
9
9
9
9
9
9
9
9
9
9
9
9
27
9
9
9
9
18
9
9
9
16
18
15
9
9
9
9
9
9
17
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
18
34
9
9
9
9
9
9
9
18
9
9
9
9


In [10]:
full_person = imdb2.get_person('0815495')

In [11]:
full_person

<Person id:0815495[http] name:_Rangel, Arturo Soto_>